In [1]:
# Define IAM role
import boto3
import re
import os
import sagemaker
import json

sagemaker.config INFO - Not applying SDK defaults from location: /opt/homebrew/share/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/anupam/Library/Application Support/sagemaker/config.yaml


In [2]:
sess = sagemaker.Session()
config=json.load(open("../.keys.json"))
role = config["SAGEMAKER_ARN"] # Also added EC2 access for accessing custom image

sagemaker.config INFO - Not applying SDK defaults from location: /opt/homebrew/share/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/anupam/Library/Application Support/sagemaker/config.yaml


In [3]:
# Data should be downloaded inside docker container.

image = config["TRAINING_IMAGE"]

nn_model = sagemaker.estimator.Estimator(
    base_job_name="nn-model-dummy",
    image_uri=image,
    role=role,
    instance_count=1,
    instance_type="ml.m4.xlarge",
    output_path=f"s3://dataforml/model-training/output",
    sagemaker_session=sess,
    input_mode="File",
    hyperparameters={
        "epochs":2
    }
)

In [4]:
training_input=sagemaker.estimator.TrainingInput(s3_data="s3://dataforml/preprocessed/")

In [6]:
nn_model.fit({"train":training_input})

INFO:sagemaker:Creating training-job with name: nn-model-dummy-2023-12-11-15-26-04-693


2023-12-11 15:26:07 Starting - Starting the training job...
2023-12-11 15:26:32 Starting - Preparing the instances for training.........
2023-12-11 15:28:17 Downloading - Downloading input data...
2023-12-11 15:28:40 Training - Downloading the training image............
2023-12-11 15:31:32 Uploading - Uploading generated training model[I 2023-12-11 15:31:23,121] A new study created in memory with name: no-name-7b7d383c-1f87-45ae-96d6-da5ff774494a
Seed set to 42
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/usr/local/lib/python3.10/dist-packages/lightning/pytorch/trainer/connectors/logger_connector/logger_connector.py:67: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorbo

In [ ]:
s3=boto3.client("s3") # Local dev me has S3 access. Could download file.
# s3.download_file("dataforml","preprocessed/y.csv","rn.csv")

In [ ]:
Need to write out metrics to Sagemaker